## Imports

In [12]:
# !pip install transformers==4.1.1

In [13]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from transformers import AutoConfig, AutoTokenizer, AutoModel, BertForSequenceClassification
from sklearn.metrics import classification_report
import torch

# Prepare the data


* `X` = `tweet` (text)
* `y` = `hate_speech` (label)

In [14]:
df = pd.read_excel('labeled_data.xlsx')

df = df.dropna()
X = df['tweet'].tolist()
y = df['hate_speech'].to_list()

## Creating Test dataset (20%)
train_texts, test_texts, train_labels, test_labels = train_test_split(X, y, random_state=0, test_size=.2)
## Create Valid dataset (20%)
train_texts, valid_texts, train_labels, valid_labels = train_test_split(train_texts, train_labels, random_state=0, test_size=.2)

## Loading model and tokenizer 

In [15]:
config = AutoConfig.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=8)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [16]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=200)
val_encodings = tokenizer(valid_texts, truncation=True, padding=True, max_length=200)

## PyTorch dataset object generation

In [17]:
class ClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        # initialization
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # slicing method X[index]
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ClassificationDataset(train_encodings, train_labels)
valid_dataset = ClassificationDataset(val_encodings, valid_labels)

## Training

In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=36,  # batch size per device during training
    per_device_eval_batch_size=36,   # batch size for evaluation
    learning_rate= 5e-6,
    do_eval=True,
    evaluation_strategy = 'epoch',
    output_dir='./results'
)

trainer = Trainer(    
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset

)

trainer.train()

Epoch,Training Loss,Validation Loss


## Predict and evaluate

In [ ]:
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=50)
test_dataset = ClassificationDataset(test_encodings, test_labels)

In [ ]:
pred = trainer.predict(test_dataset)
print(classification_report(pred.label_ids, pred.predictions.argmax(-1)))